In [8]:
import os
import glob
import gzip
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm

import shapely.wkt as wkt
from shapely.geometry import Point, LineString, box
from shapely.ops import nearest_points
import lxml.etree as ET
import network_io as nio
from itertools import combinations
import random

districts = gpd.read_file("../../../../data/visualisation/districts_paris.geojson")

In [9]:
is_for_1pm = False

if is_for_1pm:
    file_path = '../../../../data/pop_1pm/paris_1pm_network.xml.gz'
    base_output_links_no_policies = '../../../../data/pop_1pm/output_1pm/'
    output_dir = '../../../../data/pop_1pm_policies_in_relevant_zones/'

else:
    file_path = '../../../../data/pop_1pct/paris_1pct_network.xml.gz'
    base_output_links_no_policies = '../../../../data/pop_1pct/output_1pct/'
    output_dir = '../../../../data/pop_1pct_policies_in_relevant_zones/'

base_network_no_policies = nio.read_network_data(base_output_links_no_policies)
base_network_no_policies.rename(columns={'link': 'id'}, inplace=True)

# Parse nodes and edges
nodes_dict = nio.parse_nodes(file_path)
df_edges = nio.parse_edges(file_path, nodes_dict)
gdf_edges = gpd.GeoDataFrame(df_edges, geometry='geometry', crs='EPSG:2154')
gdf_edges_4326 = gdf_edges.to_crs(epsg=4326)

# Spatial join to find districts overlapping with each edge
gdf_edges_with_districts = gpd.sjoin(gdf_edges_4326, districts, how='left', op='intersects')

# Group by edge and aggregate the district names
gdf_edges_with_districts = gdf_edges_with_districts.groupby('id').agg({
    'from': 'first',
    'to': 'first',
    'length': 'first',
    'freespeed': 'first',
    'capacity': 'first',
    'permlanes': 'first',
    'oneway': 'first',
    'modes': 'first',
    'geometry': 'first',
    'c_ar': lambda x: list(x.dropna())
}).reset_index()

# Rename the aggregated column to 'district'
gdf_edges_with_districts.rename(columns={'c_ar': 'district'}, inplace=True)

# Convert freespeed and capacity to numeric values
gdf_edges_with_districts['freespeed'] = pd.to_numeric(gdf_edges_with_districts['freespeed'], errors='coerce')
gdf_edges_with_districts['capacity'] = pd.to_numeric(gdf_edges_with_districts['capacity'], errors='coerce')

gdf_edges_with_highway = gdf_edges_with_districts.merge(base_network_no_policies[['id', 'osm:way:highway']], on='id', how='left')

/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [10]:
higher_order_roads = ['tertiary', 'secondary', 'primary', 'secondary_link', 'primary_link', 'tertiary_link']
filtered_gdf = gdf_edges_with_highway[gdf_edges_with_highway['osm:way:highway'].isin(higher_order_roads)]
filtered_gdf = gpd.GeoDataFrame(filtered_gdf, geometry='geometry', crs='EPSG:4326')
gdf_edges_with_highway = gpd.GeoDataFrame(gdf_edges_with_highway, geometry='geometry', crs='EPSG:4326')

In [11]:
# # Create a larger plot with thinner lines
# fig, ax = plt.subplots(figsize=(15, 15))
# gdf_edges_with_highway.plot(ax=ax, linewidth=0.5, color='grey', label = "Network")
# filtered_gdf.plot(ax=ax, linewidth=0.5, color = "blue", label = "Higher order roads")

# # Customize the plot (optional)
# plt.title('Streets on which to apply the policies')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.legend()
# plt.show()

## Amend here the number of subsets that we want to create. 

In [12]:
relevant_subsets = set()

for i in range(1, 21):
    relevant_subsets.add((i))
    
relevant_subsets.add((1,2,3,4))
relevant_subsets.add((5,6,7))

In [13]:
dataframes = {}
for combination in relevant_subsets:
    # print(combination)
    if isinstance(combination, int):
        combination = (combination,)
    
    df_copy = gdf_edges_with_highway.copy()
    df_copy['policy_introduced'] = df_copy['district'].apply(
        lambda districts: any(d in districts for d in combination)
    ) & df_copy.apply(
        lambda row: 'car' in row['modes'] and row['osm:way:highway'] in higher_order_roads, axis=1
    )
    # Modify freespeed and capacity based on the policy_introduced condition
    df_copy.loc[df_copy['policy_introduced'], 'capacity'] = df_copy.loc[df_copy['policy_introduced'], 'capacity'] / 2
    dataframes[combination]  = df_copy
    # print(df_copy['policy_introduced'].value_counts())

In [15]:
# Directory to save the files
os.makedirs(output_dir, exist_ok=True)

# Create and save the networks
for key, df in dataframes.items():
    # Determine the filename based on the combination
    combination = key
    if len(combination) == 1:
        filename = f"network_d_{combination[0]}.xml.gz"
    else:
        filename = f"network_d_{'_'.join(map(str, combination))}.xml.gz"
    
    # Convert the DataFrame back to XML
    xml_tree = nio.dataframe_to_xml(df, nodes_dict)
    
    # Write the XML to a compressed .gz file
    file_path = os.path.join(output_dir, filename)
    nio.write_xml_to_gz(xml_tree, file_path)

# Example: Display the file paths of the saved files
output_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir)]